<a href="https://colab.research.google.com/github/Rana-ob/learn--for-GP/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# تأكد إنك غيرت نوع وقت التشغيل إلى GPU (Runtime -> Change runtime type -> GPU)

# إلغاء تثبيت bitsandbytes لضمان عدم وجود أي نسخ قديمة (فقط للتنظيف)
!pip uninstall -y bitsandbytes bitsandbytes-cuda110

# تثبيت مكتبات transformers, accelerate, gradio
!pip install transformers accelerate gradio

print("تم تثبيت المكتبات الأساسية. سنتعامل مع النموذج بدون bitsandbytes.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [19]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

# اسم النموذج الجديد الذي لا يتطلب موافقة
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# ****** التعديل هنا: لن نستخدم BitsAndBytesConfig مباشرة ******
# بدلاً من ذلك، سنحدد dtype مباشرة لضمان التحميل على GPU بفاعلية
# إعدادات Quantization (لتقليل استهلاك الذاكرة وتشغيله على GPU Colab)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# تحميل الـ Tokenizer والنموذج
print(f"جاري تحميل النموذج {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config, # <--- تم إزالة هذا السطر
    torch_dtype=torch.float16, # <--- هذا السطر الجديد سيجعل النموذج أصغر ويتحمل على الـ GPU
    device_map="auto" # بيحمل النموذج على الـ GPU تلقائياً
)
print(f"تم تحميل النموذج على الجهاز: {model.device}")
print("جاهز للبدء!")


# الأسئلة والأجوبة اللي تبغى البوت يركز عليها
initial_qa_examples = [
    {"role": "user", "content": "هل الأرباح تعاد استثمارها تلقائيا؟"},
    {"role": "assistant", "content": "نعم، يعاد استثمار الأرباح تلقائيًا لتعظيم العائد طويل الأجل."},
    {"role": "user", "content": "هل الاستثمار مضمون؟"},
    {"role": "assistant", "content": "لا يوجد استثمار مضمون 100٪، لكننا نحرص على تقليل المخاطر من خلال التنويع والاستثمار في أدوات شرعية وآمنة."},
    {"role": "user", "content": "ما هو حكم الاستثمار بأموال اليتيم؟"},
    {"role": "assistant", "content": "حكم استثمار أموال اليتيم مستحب والدليل من القرآن الكريم: ﴿وَلَا تَقْرَبُوا مَالَ الْيَتِيمِ إِلَّا بِالَّتِي هِيَ أَحْسَنُ﴾ [الأنعام:152]. وقد فسر العلماء ذلك بأن التي هي أحسن هي التجارة فيها والتسبب فيها حتى تنمو."},
    {"role": "user", "content": "هل يسمح بالسحب من حساب اليتيم اذا وصل للعمر القانوني 18؟"},
    {"role": "assistant", "content": "نعم، يسمح بالسحب بشكل جزئي كحد أقصى خمسون ألف ريال سنوياً إلى أن يبلغ عمر الـ 21."},
]

# دالة الدردشة اللي Gradio بتستخدمها
def gemma_chatbot(message, history):
    # تحويل سجل المحادثة من Gradio ليتناسب مع نموذج TinyLlama (المشابه لـ ChatML)
    # نستخدم 'messages' type في Gradio Chatbot ليتوافق مع تنسيق الأدوار
    chat_messages = [{"role": "system", "content": "أنت مساعد ذكي ومفيد وودود متخصص في الاستثمارات الشرعية وأموال الأيتام. أجب بدقة على الأسئلة المتعلقة بالاستثمار وأموال الأيتام."}]

    # إضافة الأمثلة المحددة لسياق المحادثة
    for qa in initial_qa_examples:
        chat_messages.append(qa)

    # إضافة تاريخ المحادثة الحالي
    # Gradio الآن يرسل history كقائمة من القواميس إذا استخدمت type='messages'
    for item in history:
        chat_messages.append({"role": item["role"], "content": item["content"]})

    # إضافة رسالة المستخدم الحالية
    chat_messages.append({"role": "user", "content": message})

    # تجهيز المدخلات للنموذج ونقلها للـ GPU
    input_ids = tokenizer.apply_chat_template(
        chat_messages,
        add_generation_prompt=True, # مهم عشان يبدأ النموذج يجاوب
        return_tensors="pt"
    ).to(model.device)

    # توليد الإجابة من نموذج
    outputs = model.generate(
        input_ids,
        max_new_tokens=500, # أقصى عدد كلمات جديدة ممكن يولدها البوت
        do_sample=True,
        temperature=0.7,    # للتحكم في عشوائية الإجابة
        top_p=0.9,          # للتحكم في تنوع الإجابة
        pad_token_id=tokenizer.eos_token_id, # مهم لتجنب رموز نهاية غير مرغوبة
    )

    # فك تشفير الإجابة وعرضها (الجزء اللي تولّد جديد فقط)
    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    return response_text

# إعداد واجهة Gradio
iface = gr.ChatInterface(
    gemma_chatbot,
    chatbot=gr.Chatbot(height=500, type='messages'), # <--- تم إضافة type='messages' هنا
    textbox=gr.Textbox(placeholder="اكتب سؤالك هنا...", container=False, scale=7),
    title="بوت الاستثمار الشرعي وأموال الأيتام (مدعوم بـ TinyLlama)",
    description="اسألني عن استثمار الأرباح، ضمان الاستثمار، حكم استثمار أموال اليتيم، أو شروط السحب.",
    theme="soft",
    submit_btn="إرسال",
    # clear_btn="مسح المحادثة", # <--- تم حذف هذا السطر
    examples=[
        "هل الأرباح تعاد استثمارها؟",
        "هل الاستثمار مضمون؟",
        "ما هو حكم الاستثمار بأموال اليتيم؟",
        "هل يسمح بالصرف من حساب اليتيم بعد 18 سنة؟"
    ]
)

# تشغيل الواجهة
iface.queue().launch(share=True)

PackageNotFoundError: No package metadata was found for bitsandbytes